# MOT Test Data Generation

This notebook generates synthetic MOT test data for analysis. The assignment requires:
  - Tabular data with at least 500 rows
  - Non-tabular data generated with AI
  - Explanation of generation and field distributions

**Author:** Gabriel Guimaraes

**Date:** 6.11.25

**Module:** Practical Data Analytics Level 5

## 1. Import required libraries

We'll use:
  - **pandas**: Data manipulation and `DataFrames`
  - **numpy**: Numerical operations and random number generation
  - **faker**: Generate realistic fake data (names, addresses, dates, etc.)
  - **sqlite3/sqlalchemy**: Database creation and management
  - **datetime**: Date handeling

In [1]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random
import sqlite3
from sqlalchemy import create_engine

fake = Faker('en_GB')
np.random.seed(16)  # For reproducibility

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Define Data Generation Parameters

Setting up the scale of our synthetic dataset:
  - **500 vehicles** (minimum requirement)
  - **1000 MOT tests** (vehicles can have multiple tests)
  - **20 test centres** across UK regions
  - **50 examiners** conducting tests

We also define realistic distributions for vehicle makes, fuel types, and regions based on UK MOT statistics.

In [10]:
# Define Data Generation Parameters
NUM_VEHICLES = 500
NUM_TESTS = 1000
NUM_TEST_CENTRES = 20
NUM_EXAMINERS = 50

# Vehicle makes and models 
VEHICLE_MAKES = ['Ford', 'Vauxhall', 'Volkswagen', 'BMW', 'Mercedes', 'Audi', 'Toyota', 'Nissan']
VEHICLE_MODELS = {
  'Ford': ['Fiesta', 'Focus', 'Mondeo', 'Kuga', 'Transit'],
  'Vauxhall': ['Corsa', 'Astra', 'Insignia', 'Mokka'],
  'Volkswagen': ['Golf', 'Polo', 'Passat', 'Tiguan'],
  'BMW': ['3 Series', '5 Series', 'X3', '1 Series'],
  'Mercedes': ['C-Class', 'E-Class', 'A-Class', 'GLC'],
  'Audi': ['A3', 'A4', 'Q3', 'Q5'],
  'Toyota': ['Yaris', 'Corolla', 'RAV4', 'Aygo'],
  'Nissan': ['Qashqai', 'Juke', 'Micra', 'Leaf']
}

FUEL_TYPES = ['Petrol', 'Diesel', 'Electric', 'Hybrid']
UK_REGIONS = ['London', 'South East', 'North West', 'Midlands', 'Scotland', 'Wales', 'South West', 'North East', 'Yorkshire']
DEFECT_CATEGORIES = ['Brakes', 'Tyres', 'Lights', 'Suspension', 'Steering', 'Exhaust', 'Body', 'Registration Plates', 'Seatbelts']
DEFECT_SEVERITIES = ['DANGEROUS', 'MAJOR', 'MINOR', 'ADVISORY']

print("Data generation parameters defined.")


Data generation parameters defined.


## 3. Generate Vehicle Data 

Creating the **vehicles** table with 500 records. Each vehicle has:
  - Unique vehicle_id and registration plate
  - Make and model from realistic UK distributions
  - Fuel type (weighted: Petrol 45%, Diesel 40%, Hybrid 10%, Electric 5%)
  - Registration date between 2010-2023
  - Vehicle class (most are class 4 - standard cars. subtypes: A-J)

This represents the foundational data for our MOT analysis.

In [30]:
def generate_vehicles(num_vehicles) -> pd.DataFrame:
    """Generate synthetic vehicle data."""
    vehicles = []

    for i in range(num_vehicles):
        make = random.choice(VEHICLE_MAKES)
        model = random.choice(VEHICLE_MODELS[make])

        registration_date = fake.date_between(start_date='-15y', end_date='-2y')
        registration = fake.license_plate()

        # Data: https://assets.publishing.service.gov.uk/media/66bdf9923cc0741b923146e1/ntsq09029.ods
        fuel_type = np.random.choice(FUEL_TYPES, p=[0.61, 0.31, 0.02, 0.06])  # Weighted probabilities base on 2023 data
        vehicle_class = '4'  # Most vehicles are class 4 (standard cars)

        vehicle = {
            'vehicle_id': i + 1,
            'registration': registration,
            'make': make,
            'model': model,
            'fuel_type': fuel_type,
            'registration_date': registration_date,
            'vehicle_class': vehicle_class 
        }

        vehicles.append(vehicle)


    return pd.DataFrame(vehicles)

vehicle_df = generate_vehicles(NUM_VEHICLES)
print(f"Generated {len(vehicle_df)} vehicles.")
vehicle_df.head()
        

Generated 500 vehicles.


,vehicle_id,registration,make,model,fuel_type,registration_date,vehicle_class
0,1,QN75SUA,Audi,A3,Petrol,2015-12-14,4
1,2,WW34IPX,Volkswagen,Passat,Petrol,2022-06-04,4
2,3,TN38APZ,Ford,Fiesta,Petrol,2020-01-11,4
3,4,OU47 QGQ,Toyota,Corolla,Petrol,2016-10-15,4
4,5,CF22QMA,Volkswagen,Golf,Petrol,2016-07-24,4
